In [1]:
from queue import PriorityQueue
# Heuristic function to estimate the cost of reaching the goal state from the current state
def heuristic(state):
    m_left, c_left, b_pos, m_right, c_right = state
    return (m_left + c_left - 2) // 2 + (m_right + c_right - 2) // 2
# Function to check if a state is valid
def is_valid(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if m_left < 0 or c_left < 0 or m_right < 0 or c_right < 0:
        return False
    if m_left > 3 or c_left > 3 or m_right > 3 or c_right > 3:
        return False
    if (c_left > m_left > 0) or (c_right > m_right > 0):
        return False
    return True
# Function to generate the next valid states from the current state
def next_states(state):
    m_left, c_left, b_pos, m_right, c_right = state
    if b_pos == 'left':
        moves = [(2, 0), (0, 2), (1, 1), (1, 0), (0, 1)]
        next_states = [(m_left-m, c_left-c, 'right', m_right+m, c_right+c) for m, c in moves]
    else:
        moves = [(-2, 0), (0, -2), (-1, -1), (-1, 0), (0, -1)]
        next_states = [(m_left+m, c_left+c, 'left', m_right-m, c_right-c) for m, c in moves]
    return [state for state in next_states if is_valid(state)]
# A* Search algorithm
def astar(start_state):
    frontier = PriorityQueue()  # Priority queue to explore states based on the sum of cost and heuristic value
    frontier.put((0, [start_state]))  # Insert the initial state with cost 0
    explored = set()    
    while not frontier.empty():
        cost, path = frontier.get()  # Get the state with lowest cost from the priority queue
        current_state = path[-1]        
        if current_state == (0, 0, 'right', 3, 3):
            return path        
        if current_state not in explored:
            explored.add(current_state)
            for next_state in next_states(current_state):
                new_cost = len(path) - 1  # Uniform cost for each move
                new_path = path + [next_state]
                frontier.put((new_cost + heuristic(next_state), new_path))  # Insert the new state with its priority into the priority queue
    return None
# Testing the algorithm with the initial state (3, 3, 'left', 0, 0)
start_state = (3, 3, 'left', 0, 0)
path = astar(start_state)
# Printing the path from the initial state to the goal state
if path:
    for state in path:
        print(state)
else:
    print("No solution found.")

(3, 3, 'left', 0, 0)
(2, 2, 'right', 1, 1)
(0, 2, 'left', 3, 1)
(0, 0, 'right', 3, 3)


In [13]:
import heapq

# Heuristic function: number of people left on the starting side
def heuristic(state):
    left_missionaries, left_cannibals, _, _, _ = state
    return left_missionaries + left_cannibals  # Total people left to cross

# Generate valid moves from a given state
def get_possible_moves(state):
    left_missionaries, left_cannibals, right_missionaries, right_cannibals, boat_position = state
    possible_moves = []

    # Define possible transitions
    transitions = [
        (1, 0),  # One missionary
        (2, 0),  # Two missionaries
        (0, 1),  # One cannibal
        (0, 2),  # Two cannibals
        (1, 1),  # One missionary and one cannibal
    ]

    # Generate new states from valid transitions
    for transition in transitions:
        m, c = transition  # Missionaries and cannibals to move
        if boat_position == 'left':
            new_left_m = left_missionaries - m
            new_left_c = left_cannibals - c
            new_right_m = right_missionaries + m
            new_right_c = right_cannibals + c
            new_boat_position = 'right'
        else:
            new_left_m = left_missionaries + m
            new_left_c = left_cannibals + c
            new_right_m = right_missionaries - m
            new_right_c = right_cannibals - c
            new_boat_position = 'left'

        # Ensure the move doesn't violate the constraints
        if (
            new_left_m >= 0
            and new_left_c >= 0
            and new_right_m >= 0
            and new_right_c >= 0
            and (new_left_m == 0 or new_left_m >= new_left_c)
            and (new_right_m == 0 or new_right_m >= new_right_c)
        ):
            possible_moves.append(
                (
                    new_left_m,
                    new_left_c,
                    new_right_m,
                    new_right_c,
                    new_boat_position,
                )
            )

    return possible_moves

# A* Search for solving the Missionaries and Cannibals problem
def astar(start, goal):
    # Priority queue to track states with combined cost and heuristic
    queue = []
    heapq.heappush(queue, (0, 0, start, [start]))  # (total_cost, path_cost, state, path)

    visited = set()  # Set to track visited states to avoid cycles

    while queue:
        total_cost, path_cost, current_state, path = heapq.heappop(queue)

        if current_state == goal:
            return path  # Solution found, return the path

        if current_state not in visited:
            visited.add(current_state)

            # Explore possible moves
            for new_state in get_possible_moves(current_state):
                if new_state not in visited:
                    new_path_cost = path_cost + 1  # Increment path cost
                    total_cost = new_path_cost + heuristic(new_state)  # A* combines path cost and heuristic
                    new_path = path + [new_state]
                    heapq.heappush(queue, (total_cost, new_path_cost, new_state, new_path))

    return None  # No solution found

# Example usage
if __name__ == "__main__":
    # Starting state: All missionaries and cannibals on the left
    start_state = (3, 3, 0, 0, 'left')
    # Goal state: All missionaries and cannibals on the right
    goal_state = (0, 0, 3, 3, 'right')

    # Use A* Search to find a solution
    solution = astar(start_state, goal_state)

    if solution:
        print("A* Search Solution:")
        for step, state in enumerate(solution):
            print(f"Step {step}: {state}")
    else:
        print("No solution found.")


A* Search Solution:
Step 0: (3, 3, 0, 0, 'left')
Step 1: (2, 2, 1, 1, 'right')
Step 2: (3, 2, 0, 1, 'left')
Step 3: (3, 0, 0, 3, 'right')
Step 4: (3, 1, 0, 2, 'left')
Step 5: (1, 1, 2, 2, 'right')
Step 6: (2, 2, 1, 1, 'left')
Step 7: (0, 2, 3, 1, 'right')
Step 8: (0, 3, 3, 0, 'left')
Step 9: (0, 1, 3, 2, 'right')
Step 10: (0, 2, 3, 1, 'left')
Step 11: (0, 0, 3, 3, 'right')
